In [2]:
import random
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.nn import Flatten

transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              Flatten(0)
                              ])
trainset = datasets.MNIST('./data', download=True, train=True, transform=transform)

In [3]:
train_ids = random.sample(range(0, 50000), 10000 * 2)

In [76]:
from torch.utils.data import DataLoader, Dataset
import random

class SumBalanceDataset(Dataset):
    def __init__(self, dataset, digit_ids):
        digit_to_id = {}
        
        for d in range(10):
            digit_to_id[d] = []
        
        for d_id in digit_ids:
            digit_to_id[dataset[d_id][1]].append(d_id)
        
        self.num_train = len(digit_ids) // 2
        self.digit_to_id = digit_to_id
        self.dataset = dataset
        
        self.digit_id_pairs = self.build_balanced_sum()
    
    def build_balanced_sum(self):
        digit_id_pairs = []
        
        for i in range(self.num_train):
            s = random.randint(0, 18)
            d0, d1 = self.sample_digits_from_sum(s)
            
            d0_id = random.sample(self.digit_to_id[d0], 1)[0]
            d1_id = random.sample(self.digit_to_id[d1], 1)[0]
            
            s_ids = [random.sample(self.digit_to_id[s % 10], 1)[0]]
            if s >= 10:
                s_ids.insert(0, random.sample(self.digit_to_id[s // 10], 1)[0])
            
            digit_id_pairs.append((d0_id, d1_id, d0, d1, s_ids))
        
        return digit_id_pairs

    def build_image(self, digit_ids):
        final = -1 * torch.ones((1, 56, 56))
        
        assert len(digit_ids) == 1 or len(digit_ids) == 2
        
        if len(digit_ids) == 1:
            final[:, 14:42, 14:42] = self.dataset[digit_ids[0]][0].reshape((1, 28, 28))
        
        elif len(digit_ids) == 2:
            final[:, 14:42, :28] = self.dataset[digit_ids[0]][0].reshape((1, 28, 28))
            final[:, 14:42, 28:] = self.dataset[digit_ids[1]][0].reshape((1, 28, 28))
        
        return final
    
    def sample_digits_from_sum(self, sum_val):
        d0 = random.randint(max(0, sum_val - 9), min(sum_val, 9))
        d1 = sum_val - d0
        return (d0, d1)

    def __len__(self):
        return self.num_train
    
    def __getitem__(self, idx):
        d0_id, d1_id, d0, d1, s_ids = self.digit_id_pairs[idx]
        
        d0_image = self.build_image([d0_id])
        d1_image = self.build_image([d1_id])
        
        s_image = self.build_image(s_ids)
        
        return {
            'pixels': torch.unsqueeze(torch.stack((d0_image, d1_image, s_image), dim=0), dim=0),
            'summation': [d0 + d1],
            'digit0': [d0],
            'digit1': [d1]
        }

train_balanced = SumBalanceDataset(trainset, train_ids)

trainloader = DataLoader(
    train_balanced,
    shuffle=False
)

In [99]:
class Net(torch.nn.Module):
    def __init__(self, num_digits):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)

        self.pool = nn.MaxPool2d(2, 2)

        self.lin1 = nn.Linear(16 * 11 * 11, 512)
        self.lin2 = nn.Linear(512, num_digits)

        self.relu = nn.ReLU()

        self.drop = nn.Dropout(p=0.5)

    def forward(self, x):
        x = torch.squeeze(x, dim=0)

        x = x.reshape(3, 1, 56, 56)

        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)

        x = x.reshape(3, -1)

        x = self.lin1(x)
        x = self.relu(x)

        x = self.drop(x)

        y_digit = self.lin2(x)

        return torch.unsqueeze(y_digit, dim=0)

In [100]:
class SumLayerExplicit(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, digit0, digit1):
        #x = F.softmax(x, dim=2)

        #digit0, digit1 = torch.squeeze(x, dim=0)

        digit0 = F.softmax(digit0, dim=1)
        digit1 = F.softmax(digit1, dim=1)

        digit0 = torch.reshape(digit0, (10, 1))
        digit1 = torch.reshape(digit1, (1, 10))
        d = torch.matmul(digit0, digit1)
        d = d.repeat(1, 1, 1, 1)
        f = torch.flip(torch.eye(10), dims=(0,)).repeat(1, 1, 1, 1)
        conv_diag_sums = F.conv2d(d, f, padding=(9, 0), groups=1)[..., 0]

        return torch.squeeze(conv_diag_sums, dim=0)

In [101]:
from torch import optim
from torch.nn import BCEWithLogitsLoss, BCELoss
import itertools


net_10 = Net(10)
net_19 = Net(19)

sum_net = SumLayerExplicit()

opt = optim.Adam(itertools.chain(net.parameters(), sum_net.parameters()), lr=0.001)

loss = torch.nn.L1Loss()

In [103]:
net.train()
sum_net.train()

for d in trainloader:
    opt_sum.zero_grad()
    
    x = d['pixels']
    
    digit_logits = net_10(x[:, :2])
    sum_logits = net_19(x[:, 0])
    
    digit_sum = sum_net(digit_logits[:, 0, :10], digit_logits[:, 1, :10])
    
    sum_out = F.softmax(sum_logits[:, 2], dim=1)
    
    diff = torch.cdist(digit_sum, sum_out)
    
    diff_loss = loss(diff, torch.zeros((1, 1)))
    
    print(diff_loss)
    print(torch.argmax(digit_logits, dim=2))
    print(torch.argmax(digit_sum))
    
    diff_loss.backward()

    opt.step()

tensor(0.1166, grad_fn=<L1LossBackward>)
tensor([[1, 2, 1]])
tensor(9)
tensor(0.1237, grad_fn=<L1LossBackward>)
tensor([[1, 3, 0]])
tensor(9)
tensor(0.1237, grad_fn=<L1LossBackward>)
tensor([[1, 2, 3]])
tensor(9)
tensor(0.1185, grad_fn=<L1LossBackward>)
tensor([[2, 8, 9]])
tensor(9)
tensor(0.1168, grad_fn=<L1LossBackward>)
tensor([[9, 4, 3]])
tensor(9)
tensor(0.1202, grad_fn=<L1LossBackward>)
tensor([[0, 0, 1]])
tensor(9)
tensor(0.1185, grad_fn=<L1LossBackward>)
tensor([[4, 0, 4]])
tensor(9)
tensor(0.1258, grad_fn=<L1LossBackward>)
tensor([[3, 2, 0]])
tensor(9)
tensor(0.1236, grad_fn=<L1LossBackward>)
tensor([[3, 4, 3]])
tensor(9)
tensor(0.1191, grad_fn=<L1LossBackward>)
tensor([[3, 3, 3]])
tensor(9)
tensor(0.1133, grad_fn=<L1LossBackward>)
tensor([[0, 3, 1]])
tensor(9)
tensor(0.1230, grad_fn=<L1LossBackward>)
tensor([[3, 1, 3]])
tensor(9)
tensor(0.1167, grad_fn=<L1LossBackward>)
tensor([[3, 9, 3]])
tensor(9)
tensor(0.1188, grad_fn=<L1LossBackward>)
tensor([[3, 3, 3]])
tensor(9)
tensor

tensor(0.1220, grad_fn=<L1LossBackward>)
tensor([[9, 3, 0]])
tensor(9)
tensor(0.1190, grad_fn=<L1LossBackward>)
tensor([[2, 3, 0]])
tensor(9)
tensor(0.1208, grad_fn=<L1LossBackward>)
tensor([[1, 1, 2]])
tensor(9)
tensor(0.1176, grad_fn=<L1LossBackward>)
tensor([[1, 9, 4]])
tensor(9)
tensor(0.1220, grad_fn=<L1LossBackward>)
tensor([[3, 3, 0]])
tensor(9)
tensor(0.1152, grad_fn=<L1LossBackward>)
tensor([[1, 2, 1]])
tensor(9)
tensor(0.1223, grad_fn=<L1LossBackward>)
tensor([[3, 2, 0]])
tensor(9)
tensor(0.1173, grad_fn=<L1LossBackward>)
tensor([[3, 9, 1]])
tensor(9)
tensor(0.1178, grad_fn=<L1LossBackward>)
tensor([[0, 9, 4]])
tensor(9)
tensor(0.1172, grad_fn=<L1LossBackward>)
tensor([[3, 3, 3]])
tensor(9)
tensor(0.1243, grad_fn=<L1LossBackward>)
tensor([[4, 3, 8]])
tensor(9)
tensor(0.1184, grad_fn=<L1LossBackward>)
tensor([[3, 1, 3]])
tensor(9)
tensor(0.1160, grad_fn=<L1LossBackward>)
tensor([[8, 1, 0]])
tensor(9)
tensor(0.1176, grad_fn=<L1LossBackward>)
tensor([[8, 4, 3]])
tensor(9)
tensor

tensor(0.1212, grad_fn=<L1LossBackward>)
tensor([[2, 8, 2]])
tensor(9)
tensor(0.1249, grad_fn=<L1LossBackward>)
tensor([[1, 3, 4]])
tensor(9)
tensor(0.1166, grad_fn=<L1LossBackward>)
tensor([[4, 8, 3]])
tensor(9)
tensor(0.1222, grad_fn=<L1LossBackward>)
tensor([[0, 5, 1]])
tensor(9)
tensor(0.1161, grad_fn=<L1LossBackward>)
tensor([[1, 4, 0]])
tensor(9)
tensor(0.1158, grad_fn=<L1LossBackward>)
tensor([[3, 2, 3]])
tensor(9)
tensor(0.1230, grad_fn=<L1LossBackward>)
tensor([[3, 9, 8]])
tensor(9)
tensor(0.1191, grad_fn=<L1LossBackward>)
tensor([[1, 1, 9]])
tensor(9)
tensor(0.1264, grad_fn=<L1LossBackward>)
tensor([[0, 8, 0]])
tensor(9)
tensor(0.1223, grad_fn=<L1LossBackward>)
tensor([[8, 1, 3]])
tensor(9)
tensor(0.1250, grad_fn=<L1LossBackward>)
tensor([[8, 9, 3]])
tensor(9)
tensor(0.1163, grad_fn=<L1LossBackward>)
tensor([[3, 3, 3]])
tensor(9)
tensor(0.1186, grad_fn=<L1LossBackward>)
tensor([[8, 3, 1]])
tensor(9)
tensor(0.1209, grad_fn=<L1LossBackward>)
tensor([[4, 3, 1]])
tensor(9)
tensor

tensor(0.1181, grad_fn=<L1LossBackward>)
tensor([[3, 9, 1]])
tensor(9)
tensor(0.1233, grad_fn=<L1LossBackward>)
tensor([[1, 2, 3]])
tensor(9)
tensor(0.1199, grad_fn=<L1LossBackward>)
tensor([[2, 9, 2]])
tensor(9)
tensor(0.1225, grad_fn=<L1LossBackward>)
tensor([[2, 3, 3]])
tensor(9)
tensor(0.1187, grad_fn=<L1LossBackward>)
tensor([[3, 3, 0]])
tensor(9)
tensor(0.1185, grad_fn=<L1LossBackward>)
tensor([[3, 3, 2]])
tensor(9)
tensor(0.1137, grad_fn=<L1LossBackward>)
tensor([[9, 9, 5]])
tensor(9)
tensor(0.1220, grad_fn=<L1LossBackward>)
tensor([[3, 3, 3]])
tensor(9)
tensor(0.1241, grad_fn=<L1LossBackward>)
tensor([[3, 1, 3]])
tensor(9)
tensor(0.1182, grad_fn=<L1LossBackward>)
tensor([[2, 3, 3]])
tensor(9)
tensor(0.1217, grad_fn=<L1LossBackward>)
tensor([[0, 3, 3]])
tensor(9)
tensor(0.1195, grad_fn=<L1LossBackward>)
tensor([[2, 3, 8]])
tensor(9)
tensor(0.1188, grad_fn=<L1LossBackward>)
tensor([[9, 9, 3]])
tensor(9)
tensor(0.1216, grad_fn=<L1LossBackward>)
tensor([[2, 3, 1]])
tensor(9)
tensor

tensor(0.1248, grad_fn=<L1LossBackward>)
tensor([[3, 3, 8]])
tensor(9)
tensor(0.1166, grad_fn=<L1LossBackward>)
tensor([[9, 1, 3]])
tensor(9)
tensor(0.1201, grad_fn=<L1LossBackward>)
tensor([[3, 3, 0]])
tensor(9)
tensor(0.1172, grad_fn=<L1LossBackward>)
tensor([[3, 2, 0]])
tensor(9)
tensor(0.1181, grad_fn=<L1LossBackward>)
tensor([[4, 9, 3]])
tensor(9)
tensor(0.1233, grad_fn=<L1LossBackward>)
tensor([[3, 4, 9]])
tensor(9)
tensor(0.1228, grad_fn=<L1LossBackward>)
tensor([[3, 2, 3]])
tensor(9)
tensor(0.1172, grad_fn=<L1LossBackward>)
tensor([[9, 9, 3]])
tensor(9)
tensor(0.1177, grad_fn=<L1LossBackward>)
tensor([[3, 3, 1]])
tensor(9)
tensor(0.1261, grad_fn=<L1LossBackward>)
tensor([[9, 1, 9]])
tensor(9)
tensor(0.1208, grad_fn=<L1LossBackward>)
tensor([[3, 3, 9]])
tensor(9)
tensor(0.1227, grad_fn=<L1LossBackward>)
tensor([[9, 4, 3]])
tensor(9)
tensor(0.1202, grad_fn=<L1LossBackward>)
tensor([[0, 1, 9]])
tensor(9)
tensor(0.1195, grad_fn=<L1LossBackward>)
tensor([[1, 3, 3]])
tensor(9)
tensor

...

tensor(0.1211, grad_fn=<L1LossBackward>)
tensor([[1, 3, 3]])
tensor(9)
tensor(0.1190, grad_fn=<L1LossBackward>)
tensor([[3, 1, 1]])
tensor(9)
tensor(0.1218, grad_fn=<L1LossBackward>)
tensor([[3, 9, 9]])
tensor(9)
tensor(0.1160, grad_fn=<L1LossBackward>)
tensor([[1, 5, 8]])
tensor(9)
tensor(0.1219, grad_fn=<L1LossBackward>)
tensor([[0, 3, 8]])
tensor(9)
tensor(0.1185, grad_fn=<L1LossBackward>)
tensor([[3, 0, 2]])
tensor(9)
tensor(0.1202, grad_fn=<L1LossBackward>)
tensor([[2, 0, 1]])
tensor(9)
tensor(0.1241, grad_fn=<L1LossBackward>)
tensor([[2, 3, 3]])
tensor(9)
tensor(0.1179, grad_fn=<L1LossBackward>)
tensor([[9, 0, 1]])
tensor(9)
tensor(0.1210, grad_fn=<L1LossBackward>)
tensor([[3, 9, 2]])
tensor(9)
tensor(0.1215, grad_fn=<L1LossBackward>)
tensor([[2, 3, 4]])
tensor(9)
tensor(0.1230, grad_fn=<L1LossBackward>)
tensor([[4, 2, 3]])
tensor(9)
tensor(0.1208, grad_fn=<L1LossBackward>)
tensor([[9, 0, 9]])
tensor(9)
tensor(0.1217, grad_fn=<L1LossBackward>)
tensor([[3, 5, 3]])
tensor(9)
tensor

tensor(0.1213, grad_fn=<L1LossBackward>)
tensor([[1, 3, 4]])
tensor(9)
tensor(0.1207, grad_fn=<L1LossBackward>)
tensor([[5, 3, 3]])
tensor(9)
tensor(0.1177, grad_fn=<L1LossBackward>)
tensor([[1, 3, 3]])
tensor(9)
tensor(0.1207, grad_fn=<L1LossBackward>)
tensor([[2, 8, 0]])
tensor(9)
tensor(0.1156, grad_fn=<L1LossBackward>)
tensor([[1, 4, 9]])
tensor(9)
tensor(0.1186, grad_fn=<L1LossBackward>)
tensor([[1, 4, 0]])
tensor(9)
tensor(0.1162, grad_fn=<L1LossBackward>)
tensor([[3, 9, 2]])
tensor(9)
tensor(0.1197, grad_fn=<L1LossBackward>)
tensor([[3, 9, 0]])
tensor(9)
tensor(0.1216, grad_fn=<L1LossBackward>)
tensor([[3, 3, 3]])
tensor(9)
tensor(0.1209, grad_fn=<L1LossBackward>)
tensor([[1, 1, 8]])
tensor(9)
tensor(0.1243, grad_fn=<L1LossBackward>)
tensor([[3, 1, 3]])
tensor(9)
tensor(0.1165, grad_fn=<L1LossBackward>)
tensor([[0, 0, 0]])
tensor(9)
tensor(0.1156, grad_fn=<L1LossBackward>)
tensor([[9, 9, 9]])
tensor(9)
tensor(0.1247, grad_fn=<L1LossBackward>)
tensor([[9, 4, 0]])
tensor(9)
tensor

tensor(9)
tensor(0.1218, grad_fn=<L1LossBackward>)
tensor([[2, 3, 0]])
tensor(9)
tensor(0.1252, grad_fn=<L1LossBackward>)
tensor([[3, 4, 0]])
tensor(9)
tensor(0.1188, grad_fn=<L1LossBackward>)
tensor([[3, 3, 9]])
tensor(9)
tensor(0.1229, grad_fn=<L1LossBackward>)
tensor([[9, 1, 3]])
tensor(9)
tensor(0.1214, grad_fn=<L1LossBackward>)
tensor([[3, 9, 1]])
tensor(9)
tensor(0.1236, grad_fn=<L1LossBackward>)
tensor([[3, 3, 1]])
tensor(9)
tensor(0.1257, grad_fn=<L1LossBackward>)
tensor([[3, 1, 1]])
tensor(9)
tensor(0.1179, grad_fn=<L1LossBackward>)
tensor([[0, 3, 2]])
tensor(9)
tensor(0.1220, grad_fn=<L1LossBackward>)
tensor([[1, 2, 2]])
tensor(9)
tensor(0.1185, grad_fn=<L1LossBackward>)
tensor([[3, 3, 9]])
tensor(9)
tensor(0.1224, grad_fn=<L1LossBackward>)
tensor([[3, 1, 3]])
tensor(9)
tensor(0.1229, grad_fn=<L1LossBackward>)
tensor([[0, 3, 1]])
tensor(9)
tensor(0.1210, grad_fn=<L1LossBackward>)
tensor([[3, 3, 3]])
tensor(9)
tensor(0.1191, grad_fn=<L1LossBackward>)
tensor([[0, 8, 0]])
tensor

tensor(0.1193, grad_fn=<L1LossBackward>)
tensor([[0, 2, 2]])
tensor(9)
tensor(0.1190, grad_fn=<L1LossBackward>)
tensor([[2, 9, 2]])
tensor(9)
tensor(0.1182, grad_fn=<L1LossBackward>)
tensor([[3, 9, 1]])
tensor(9)
tensor(0.1186, grad_fn=<L1LossBackward>)
tensor([[8, 0, 3]])
tensor(9)
tensor(0.1155, grad_fn=<L1LossBackward>)
tensor([[8, 3, 3]])
tensor(9)
tensor(0.1266, grad_fn=<L1LossBackward>)
tensor([[5, 3, 2]])
tensor(9)
tensor(0.1243, grad_fn=<L1LossBackward>)
tensor([[3, 4, 9]])
tensor(9)
tensor(0.1162, grad_fn=<L1LossBackward>)
tensor([[1, 2, 1]])
tensor(9)
tensor(0.1205, grad_fn=<L1LossBackward>)
tensor([[1, 1, 1]])
tensor(9)
tensor(0.1190, grad_fn=<L1LossBackward>)
tensor([[3, 9, 9]])
tensor(9)
tensor(0.1217, grad_fn=<L1LossBackward>)
tensor([[3, 3, 1]])
tensor(9)
tensor(0.1191, grad_fn=<L1LossBackward>)
tensor([[5, 1, 3]])
tensor(9)
tensor(0.1209, grad_fn=<L1LossBackward>)
tensor([[1, 3, 3]])
tensor(9)
tensor(0.1173, grad_fn=<L1LossBackward>)
tensor([[3, 1, 2]])
tensor(9)
tensor

KeyboardInterrupt: 